In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import os

# Define paths
output_folder = 'E:/intern_work_testing/label_obj/'

for r in range(50):
    # File paths
    point_cloud_file = f'E:/intern_work_testing/converted_csv_dataset/{r:06}.csv'
    labels_file = f'E:/intern_work_testing/labels/{r:06}.csv'
    calib_file = f'E:/intern_work_testing/calib/{r:06}.csv'
    output_file = f'E:/intern_work_testing/label_obj/{r:06}.csv'

    # Load point cloud, label, and calibration data
    ptCloud_df = pd.read_csv(point_cloud_file)
    labels_df = pd.read_csv(labels_file)
    tv2c_df = pd.read_csv(calib_file, header=None)

    # Load and invert the transformation matrix Tv2c (camera to Velodyne coordinates)
    Tv2c = np.vstack([tv2c_df.values, [0, 0, 0, 1]])
    Tc2v = np.linalg.inv(Tv2c)

    # Function to transform a 3D point from camera to Velodyne coordinates
    def transform_to_velodyne(point, Tc2v):
        point_homogeneous = np.append(point, 1)  # Convert to homogeneous coordinates
        point_velodyne = np.dot(Tc2v, point_homogeneous)
        return point_velodyne[:3]

    # Convert 3D X, Y, Z coordinates in labels to Velodyne coordinates
    velodyne_coords = labels_df[['X', 'Y', 'Z']].apply(lambda row: transform_to_velodyne(row.values, Tc2v), axis=1)
    velodyne_coords = np.vstack(velodyne_coords)  # Convert to numpy array for easier access

    # Horizontal and vertical angular resolutions
    Resh = 0.17  # Horizontal angular resolution in degrees
    Resv = 0.4   # Vertical angular resolution in degrees

    # Calculate Xfront and Yfront for each point in Velodyne coordinates
    def calculate_front_view(coord):
        x, y, z = coord
        d = np.sqrt(x**2 + y**2)
        Xfront = np.arctan2(y, x) * (180 / (np.pi * Resh))  # Horizontal angle
        Yfront = np.arctan2(z, d) * (180 / (np.pi * Resv))  # Vertical angle
        return Xfront, Yfront

    front_view_coords = np.array([calculate_front_view(coord) for coord in velodyne_coords])

    # Add Xfront and Yfront columns to labels DataFrame
    labels_df['Xfront'] = front_view_coords[:, 0]
    labels_df['Yfront'] = front_view_coords[:, 1]

    # Save the updated labels DataFrame with Xfront and Yfront to a new CSV
    labels_df.to_csv(output_file, index=False)

    print(f'Successfully saved labels with front view coordinates to {output_file}')


Successfully saved labels with front view coordinates to E:/intern_work_testing/label_obj/000000.csv
Successfully saved labels with front view coordinates to E:/intern_work_testing/label_obj/000001.csv
Successfully saved labels with front view coordinates to E:/intern_work_testing/label_obj/000002.csv
Successfully saved labels with front view coordinates to E:/intern_work_testing/label_obj/000003.csv
Successfully saved labels with front view coordinates to E:/intern_work_testing/label_obj/000004.csv
Successfully saved labels with front view coordinates to E:/intern_work_testing/label_obj/000005.csv
Successfully saved labels with front view coordinates to E:/intern_work_testing/label_obj/000006.csv
Successfully saved labels with front view coordinates to E:/intern_work_testing/label_obj/000007.csv
Successfully saved labels with front view coordinates to E:/intern_work_testing/label_obj/000008.csv
Successfully saved labels with front view coordinates to E:/intern_work_testing/label_obj/0